In [27]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [28]:
openai.organization = "org-RuJeUUCVE9l16B6VH1CFTTJn"
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x11c4f7d60> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

## Check if the connection and saving is working


In [29]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7Dvz95QAUGBllf6YJ4FGdKcxscf6v at 0x11f3f74a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played at a neutral site, Globe Life Field in Arlington, Texas, due to the COVID-19 pandemic.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683555515,
  "id": "chatcmpl-7Dvz95QAUGBllf6YJ4FGdKcxscf6v",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 57,
    "total_tokens": 86
  }
}

## Start testing

In [30]:
# open our test dataframe from the csv
df = pd.read_csv('data/test/random_sample_1000.csv')
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,priceCurrency_right,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction
0,66649332,POKEMON,POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza,NaN,15.99,CAD,NaN,3227349,17589843,NaN,...,USD,NaN,2490160,66649332#17589843,0,True,"[-4.2311544418, 4.5329031944]",1,[0.0],0
1,82150767,NaN,SRAM Organic Disc Brake Pads - Level Ultimate / TLM,SRAM Organic Disc Brake Pads - Level Ultimate / TLM SRAM road disc brake pads are all-new and designed for use with SRAM road hydraulic disc brakes (HRD).Organic Compound. The pad set includes a bleed block tool for proper piston set-up.If you need any help or have any questions about the SRAM O...,22.99,GBP,NaN,350645,53413098,AVID,...,EUR,NaN,350645,82150767#53413098,1,False,"[-4.1164469719, 4.417277813]",1,[1.0],1
2,37008321,Western Digital,WD Blue 3D NAND 250GB Internal SSD - SATA III 6Gb/s M.2 2280 Solid State Drive - WDS250G2B0B,WD Blue 3D NAND 250GB Internal SSD - SATA III 6Gb/s M.2 2280 Solid State Drive - WDS250G2B0B,59.99,CAD,NaN,3065693,79398825,NaN,...,LKR,NaN,3065693,37008321#79398825,1,False,"[-4.3920178413, 4.6791453362]",1,[1.0],1
3,84336913,Shimano,Shimano Dura-Ace CS-R9100 11-25T 11sp Cassette,"Match your riding style as well as your chosen terrain with Shimano's lightweight Titanium Dura Ace CS-R9100 11-Speed Cassette. New combinations of gearing options have been added including expanded choices for gearing appropriate to wheels, climbers, and the punchers who demand the best of both...",320.99,AUD,NaN,75017,331049,NaN,...,USD,NaN,75017,84336913#331049,1,False,"[-4.372959137, 4.6612653732]",1,[1.0],1
4,6741336,Canon,Canon Canon EF-S 10-18mm f/4.5-5.6 IS STM,This item may qualify for an Instant Rebate. Any current manufacturer rebate will be refunded to your card after purchase. Please email shop@lookingglassphoto.com if you have any questions.,299.99,USD,NaN,3329047,64320932,Canon,...,ZAR,NaN,3160384,6741336#64320932,0,True,"[3.7756073475000003, -4.2413973808]",0,[0.0],0


In [31]:
df.shape

(1000, 23)

In [32]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [33]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "Do the following two product decriptions match. Answer with yes or no."},
        {"role": "user",
            "content": f" product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=2,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/1000 [00:00<?, ?it/s]

Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7f8bb9716448af02cbf7d3a3bc67ab33 in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f1775825e9dbb9d904ea8c43073b0a1b in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 4d906aeed6ecc76cd15957b95db5c75b in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 4695260d6cbf288c3cad2e6978f3651a in you

In [34]:
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
0,66649332,POKEMON,POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza,NaN,15.99,CAD,NaN,3227349,17589843,NaN,...,66649332#17589843,0,True,"[-4.2311544418, 4.5329031944]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7DvzC4pxlMgHVViGDnzMN7FFUOP6c', 'object': 'chat.completion', 'created': 1683555518, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 68, 'completion_tokens': 2, 'total_tokens': 70}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza, product 2: Pokemon S&S2: Rebel Clash BD'}]"
1,82150767,NaN,SRAM Organic Disc Brake Pads - Level Ultimate / TLM,SRAM Organic Disc Brake Pads - Level Ultimate / TLM SRAM road disc brake pads are all-new and designed for use with SRAM road hydraulic disc brakes (HRD).Organic Compound. The pad set includes a bleed block tool for proper piston set-up.If you need any help or have any questions about the SRAM O...,22.99,GBP,NaN,350645,53413098,AVID,...,82150767#53413098,1,False,"[-4.1164469719, 4.417277813]",1,[1.0],1,Yes.,"{'id': 'chatcmpl-7DvzDqC64Dum1un3IzPf7Dv4NRMDu', 'object': 'chat.completion', 'created': 1683555519, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 66, 'completion_tokens': 2, 'total_tokens': 68}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: SRAM Organic Disc Brake Pads - Level Ultimate / TLM, product 2: PASTIGLIE ORG-STEEL ROAD/LEVEL ULTIM.'}]"
2,37008321,Western Digital,WD Blue 3D NAND 250GB Internal SSD - SATA III 6Gb/s M.2 2280 Solid State Drive - WDS250G2B0B,WD Blue 3D NAND 250GB Internal SSD - SATA III 6Gb/s M.2 2280 Solid State Drive - WDS250G2B0B,59.99,CAD,NaN,3065693,79398825,NaN,...,37008321#79398825,1,False,"[-4.3920178413, 4.6791453362]",1,[1.0],1,Yes.,"{'id': 'chatcmpl-7DvzEsoD4IadKfcygeWwbHsg0BWqX', 'object': 'chat.completion', 'created': 1683555520, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 83, 'completion_tokens': 2, 'total_tokens': 85}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: WD Blue 3D NAND 250GB Internal SSD - SATA III 6Gb/s M.2 2280 Solid State Drive - WDS250G2B0B, product 2: WD BLUE 250GB M.2 SSD DRIVE'}]"
3,84336913,Shimano,Shimano Dura-Ace CS-R9100 11-25T 11sp Cassette,"Match your riding style as well as your chosen terrain with Shimano's lightweight Titanium Dura Ace CS-R9100 11-Speed Cassette. New combinations of gearing options have been added including expanded choices for gearing appropriate to wheels, climbers, and the punchers who demand the best of both...",320.99,AUD,NaN,75017,331049,NaN,...,84336913#331049,1,False,"[-4.372959137, 4.6612653732]",1,[1.0],1,Yes.,"{'id': 'chatcmpl-7DvzFsbHXd94M75AhMy8T7StCH7F2', 'object': 'chat.completion', 'created': 1683555521, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 69, 'completion_tokens': 2, 'total_tokens': 71}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Shimano Dura-Ace CS-R9100 11-25T 11sp Cassette, product 

In [35]:
# get the number of rows were the chatbot respionse is not empty
df[df['chatbot_response'] != ''].shape

(1000, 26)

In [36]:
df.tail()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
995,26200326,NaN,MultiPlus Compact 12/2000/80-30 230V VE.Bus,NaN,1.359E3,USD,NaN,4739100,77590975,NaN,...,26200326#77590975,0,False,"[3.8259320259000003, -4.2865967751]",0,[0.0],0,No.,"{'id': 'chatcmpl-7DwOO8QfStSrrju36AfmIlj2leEe1', 'object': 'chat.completion', 'created': 1683557080, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 74, 'completion_tokens': 2, 'total_tokens': 76}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: MultiPlus Compact 12/2000/80-30 230V VE.Bus, product 2: Jabra Biz 2300 USB UC Duo Binaural Headset2399-823-109'}]"
996,5727991,Kingston,Kingston Hyperx Fury Black Series,NaN,349,SEK,NaN,1877306,96813478,NaN,...,5727991#96813478,0,True,"[3.6815905571, -4.1587138176]",0,[0.0],0,Yes,"{'id': 'chatcmpl-7DwOPsi3Hkh5q6jrtIhmwnUpp516u', 'object': 'chat.completion', 'created': 1683557081, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 62, 'completion_tokens': 1, 'total_tokens': 63}, 'choices': [{'message': { ""content"": ""Yes"", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Kingston Hyperx Fury Black Series, product 2: Kingston Hyperx Fury 16GB 3200Mhz DDR4 Cl16 DIMM Black'}]"
997,95816254,SAMSUNG,Samsung Portable SSD T7 Touch USB3.2 1TB schwarz,NaN,1.6544E2,EUR,NaN,1458655,62268972,NaN,...,95816254#62268972,1,False,"[-4.3924574852, 4.6705560684]",1,[1.0],1,Yes.,"{'id': 'chatcmpl-7DwOP1L2yQ5WrUNM6MYhrG16VjAMV', 'object': 'chat.completion', 'created': 1683557081, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 78, 'completion_tokens': 2, 'total_tokens': 80}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Samsung Portable SSD T7 Touch USB3.2 1TB schwarz, product 2: Samsung Portable SSD T7 Touch MU-PC1T0K - solid state drive - 1 TB - USB 3.'}]"
998,58975591,NaN,Sony FE 70-200mm f/4.0 G OSS Lens,NaN,1398.0,USD,NaN,336149,44707698,NaN,...,58975591#44707698,0,True,"[3.7888097763, -4.2529716492]",0,[0.0],0,No.,"{'id': 'chatcmpl-7DwOQUCtSflhor28yWtTamAS4YSQf', 'object': 'chat.completion', 'created': 1683557082, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 64, 'completion_tokens': 2, 'total_tokens': 66}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Sony FE 70-200mm f/4.0 G OSS Lens, product 2: Sony FE 20mm f/1.8 G Lens'}]"
999,22275252,NaN,D1-TAPE 9MM X 7M BLACK ON WHITE IN,S0720680 Dymo,0,EUR,NaN,4559,98249213,NaN,...,22275252#98249213,0,True,"[3.82665205, -4.2858901024]",0,[0.0],0,No.,"{'id': 'chatcmpl-7DwOQ6NGHP43Y5HHfjpgOnV07UMUN', 'object': 'chat.completion', 'created': 1683557082, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 75, 'completion_tokens': 2, 'total_tokens': 77}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: D1-TAPE 9MM X 7M BLACK ON WHITE IN, product 2: Crucial SSD MX500 2.5\"" 2TB SATA III 3D Nand 7mm'}]"


In [37]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [38]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response'].apply(lambda x: clean_response(x))

In [39]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/chat_gpt/random_sample_1000/{now}_simple_promt.json')

In [40]:
# look at the wrong answers
df[df['chatbot_response_clean'] != df['label']].head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,66649332,POKEMON,POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza,NaN,15.99,CAD,NaN,3227349,17589843,NaN,...,0,True,"[-4.2311544418, 4.5329031944]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7DvzC4pxlMgHVViGDnzMN7FFUOP6c', 'object': 'chat.completion', 'created': 1683555518, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 68, 'completion_tokens': 2, 'total_tokens': 70}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza, product 2: Pokemon S&S2: Rebel Clash BD'}]",1
8,30790106,NaN,Nikon AF-S NIKKOR 50MM F1.8G,Nikon inch s AF-S Nikkor 50mm f/1.8G Lens is an attractive alternative to the classic 50mm f/1.4inch normal inchlens for everyday shooting with an FX or 35mm camera. The 50mm f/1.8 is nearly asinch fast inchas the f/1.4 (just 2/3 stops difference)and provides reduced lens flare and ghosting (plu...,388.00,NZD,NaN,730377,80988716,NaN,...,1,False,"[-3.2542202473, 3.2235870361]",1,[1.0],1,No.,"{'id': 'chatcmpl-7DvzIaOhpmR5WeK7fhOogd89RAfTD', 'object': 'chat.completion', 'created': 1683555524, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 69, 'completion_tokens': 2, 'total_tokens': 71}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Nikon AF-S NIKKOR 50MM F1.8G, product 2: Canon EF 100mm F2.8L Macro IS USM + BONUS'}]",0
42,393407,Audemars Piguet,Audemars Piguet Royal Oak Chronograph,Audemars Piguet Royal Oak,36560,USD,NaN,607611,98604797,Audemars Piguet,...,0,True,"[-4.1834125519, 4.471534729]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7Dw160gurd9V17W7haXr6mX9E4GuT', 'object': 'chat.completion', 'created': 1683555636, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 69, 'completion_tokens': 2, 'total_tokens': 71}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Audemars Piguet Royal Oak Chronograph, product 2: Audemars Piguet Royal Oak Chronograph Watch - 26331ST.OO.1220ST.02'}]",1
43,39729230,NaN,"40\"" Blaze 5 Burner Grill Cart","This Blaze stainless steel grill cart is specifically designed to house the 40\"" Blaze gas grill.The stainless steel construction ensures long-lasting durability for years of outdoor use and the four heavy duty caster wheels allow you to easily relocate the grill whenever necessary.The two stain...",514.99,USD,NaN,4325250,50547404,NaN,...,1,False,"[-4.28804636, 4.5875926018]",1,[1.0],1,No.,"{'id': 'chatcmpl-7Dw17ZS2KuRl4Ik45BTuHJBH2MXUP', 'object': 'chat.completion', 'created': 1683555637, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 72, 'completion_tokens': 2, 'total_tokens': 74}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: 40\"" Blaze 5 Burner Grill Cart, product 2: Blaze 35\"" Stainless Steel Grill Cart for 5 Burner Traditional/LTE Gas Grills BLZ-5-CART'}]",0
45,44542160,NaN,DS-7208HQHI-K2-P Hikvision 8 Channel 1080P HD-TVI Hybrid DVR (POC),Key Features8ch Power over Coax (PoC) Hybrid DVRH.265+/H.265/H.264+

In [41]:
# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")

Accuracy: 0.89


In [42]:
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

1

In [43]:
# Look at 5 complete chatbot responses dont limit the output length
# set the max_rows and max_columns options to None
# set the max_colwidth option to None
pd.set_option('display.max_colwidth', 300)

# display the first 5 chatbot responses where chatbot_response_raw is not -1
df['chatbot_response_raw'].head(3)

0    {'id': 'chatcmpl-7DvzC4pxlMgHVViGDnzMN7FFUOP6c', 'object': 'chat.completion', 'created': 1683555518, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 68, 'completion_tokens': 2, 'total_tokens': 70}, 'choices': [{'message': {
  "content": "Yes.",
  "role": "assistant"
}, 'finish_reason':...
1    {'id': 'chatcmpl-7DvzDqC64Dum1un3IzPf7Dv4NRMDu', 'object': 'chat.completion', 'created': 1683555519, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 66, 'completion_tokens': 2, 'total_tokens': 68}, 'choices': [{'message': {
  "content": "Yes.",
  "role": "assistant"
}, 'finish_reason':...
2    {'id': 'chatcmpl-7DvzEsoD4IadKfcygeWwbHsg0BWqX', 'object': 'chat.completion', 'created': 1683555520, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 83, 'completion_tokens': 2, 'total_tokens': 85}, 'choices': [{'message': {
  "content": "Yes.",
  "role": "assistant"
}, 'finish_reason':...
Name: chatbot_response_raw, dtype: object

In [44]:
# Lets look at all mistakes only show the title_left, title_right, label and chatbot_response_clean
df[df['chatbot_response_clean'] != df['label']][['title_left', 'title_right', 'label', 'chatbot_response_clean']]


,title_left,title_right,label,chatbot_response_clean
0,POKEMON POKEMON SWSH2 REBEL CLASH 3PK BLISTER - Rayquaza,Pokemon S&S2: Rebel Clash BD,0,1
8,Nikon AF-S NIKKOR 50MM F1.8G,Canon EF 100mm F2.8L Macro IS USM + BONUS,1,0
42,Audemars Piguet Royal Oak Chronograph,Audemars Piguet Royal Oak Chronograph Watch - 26331ST.OO.1220ST.02,0,1
43,"40\"" Blaze 5 Burner Grill Cart","Blaze 35\"" Stainless Steel Grill Cart for 5 Burner Traditional/LTE Gas Grills BLZ-5-CART",1,0
45,DS-7208HQHI-K2-P Hikvision 8 Channel 1080P HD-TVI Hybrid DVR (POC),DVR Hikvision TurboHD 8 canale DS-7208HQHI-K1,0,1
...,...,...,...,...
934,"Maxxis Maxxis, High Roller II, Tire, 29''x2.50, Folding, Tubeless Ready, 3C Maxx Terra, Double Down, Wide Trail, 120x2TPI, Black","MTB Plášť MAXXIS HIGH ROLLER II 27,5x2,40 3C Maxx Terra EXO TR kevlar",0,1
935,Ubiquiti UniFi AC In-Wall Access Point WiFi 5-Pack,"UAP-AC-PRO_ 5 PACK UniFi AP, AC PRO World",0,1
953,Shimano Dura-Ace CS-R9100 11-25T 11sp Cassette,Shimano Dura Ace R9100 11-Speed Cassette,0,1
967,Kingston Technology Canvas Select Plus flashgeheugen 32 GB SDHC Klasse 10 UHS-I,"Kingston 32GB microSD Canvas Select, Class 10 + SD adapter",0,1


In [45]:
# how many currencies are there in the dataset
df['priceCurrency_left'].unique()

array(['CAD', 'GBP', 'AUD', 'USD', 'TRY', 'AED', 'EUR', 'NZD', nan, 'SEK',
       'SGD', 'PLN', 'INR', 'BAM', 'DKK', 'NOK', 'VND', 'MXN', 'ZAR',
       'RON', 'RUB', 'CZK', 'MYR', 'KYD', 'лв.', 'COP', 'BRL', 'KES',
       'HRK', 'HUF', 'BGN', 'PHP', 'CHF', 'LKR', 'BDT', 'GHS', 'eur',
       'MUR', 'SAR', 'NGN'], dtype=object)

In [46]:
# How often does priceCurrency_left and priceCurrency_right match
df[df['priceCurrency_left'] == df['priceCurrency_right']].shape[0]

165